## Load Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['DISPLAY'] = ':999'
sys.path.append('/home/zachpen87/clearmap/ClearMap2/')

import dask.array as da
import fnmatch
import h5py
import holoviews as hv
import pandas as pd
from natsort import natsorted
import scipy.ndimage as ndimage
import skimage
from ClearMap.Environment import * 
import ClearMap.ImageProcessing.H5 as H5img
hv.notebook_extension('bokeh')

## Specify Directory Information

In [ ]:
directory           = '/home/zachpen87/clearmap/data/SEFL17/' 
subdirectories      = natsorted(os.listdir(directory))
subdirectories      = fnmatch.filter(subdirectories,'*ms*')

excluded_directories = ['ms_21_redo']

In [ ]:
try:
    del allcounts
except:
    pass

for subdirectory in subdirectories:
    
    if subdirectory not in excluded_directories:
        hdf5_file = os.path.join(directory, subdirectory,'data.hdf5')
        with h5py.File(hdf5_file,'a') as f:
            
            print('processing labels for: {x}'.format(x=subdirectory))
            labels = f['centroids/labels'][:]
            names = ano.convert_label(label=labels, value='name',key='id')
            acronyms = ano.convert_label(label=labels, value='acronym',key='id')
            gph_ordr = ano.convert_label(label=labels, value='graph_order',key='id')
            p_id = ano.convert_label(label=labels, value='parent_structure_id', key='id')
            p_names = np.array(['None']*len(p_id), dtype='U80')
            p_names[np.where(p_id != None)] = ano.convert_label(
                label=p_id[np.where(p_id != None)], value='name', key='id')
            pt_frame = pd.DataFrame({
                'annotation_labels' : labels,
                'annotation_names' : names,
                'annotation_abrv' : acronyms,
                'parent_labels' : p_id,
                'parent_names' : p_names,
                'graph_order' : gph_ordr})
            pt_frame.fillna('None',inplace=True) # by exluding this line, 'universe', 'brain' and 'unlabeled' will be dropped
            countsummary = pt_frame.value_counts().reset_index(name='count')
            countsummary.insert(0,'animal',subdirectory)
            countsummary.to_csv(os.path.join(directory,subdirectory,'countsummary.csv'),index=False)
            
            try:
                allcounts = pd.concat([allcounts,countsummary])
            except:
                allcounts = countsummary

allcounts.to_csv(os.path.join(directory,'results','countsummary.csv'),index=False)             